In [37]:
from openbabel import openbabel

def convert_pdb_to_pdbqt(input_pdb, output_pdbqt):
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("pdb", "pdbqt")

    # Read the PDB file
    mol = openbabel.OBMol()
    if not obConversion.ReadFile(mol, input_pdb):
        print(f"Failed to read the file {input_pdb}")
        return

    # Add hydrogens
    mol.AddHydrogens()

    # Assign Gasteiger charges
    obChargeModel = openbabel.OBChargeModel.FindType("gasteiger")
    if not obChargeModel.ComputeCharges(mol):
        print("Failed to assign Gasteiger charges.")
        return

    # Convert to PDBQT string
    pdbqt_string = obConversion.WriteString(mol)

    # Filter out unwanted lines
    filtered_pdbqt = "\n".join([line for line in pdbqt_string.splitlines()
                                if not line.startswith(("REMARK", "BRANCH", "ENDBRANCH", "ROOT", "ENDROOT", "TORSDOF", "ENDTORSDOF")) and not "UNL" in line])

    # Write the filtered PDBQT string to a file
    with open(output_pdbqt, "w") as file:
        file.write(filtered_pdbqt)

    print(f"File converted: {output_pdbqt}")

In [25]:
from rdkit import Chem
from rdkit.Chem import AllChem
import os

# Convert SMILES to 3D structure using RDKit
smiles = "CCO"  # Example SMILES
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.MMFFOptimizeMolecule(mol)

# Save the 3D structure to a PDB file
pdb_filename = "generic_ligand.pdb"
writer = Chem.PDBWriter(pdb_filename)
writer.write(mol)
writer.close()
convert_pdb_to_pdbqt("generic_ligand.pdb", "generic_ligand.pdbqt")

File converted: generic_ligand.pdbqt


In [26]:
d_block = "C1CN(CCC1(C2=CC=C(C=C2)Cl)O)CCCC(=O)C3=CC=C(C=C3)F"

from rdkit import Chem
from rdkit.Chem import AllChem
import os

# Convert SMILES to 3D structure using RDKit
smiles = d_block  # Example SMILES
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.MMFFOptimizeMolecule(mol)

# Save the 3D structure to a PDB file
pdb_filename = "d_block_ligand.pdb"
writer = Chem.PDBWriter(pdb_filename)
writer.write(mol)
writer.close()
convert_pdb_to_pdbqt("d_block_ligand.pdb", "d_block_ligand.pdbqt")

File converted: d_block_ligand.pdbqt


In [32]:
import gzip
from rdkit import Chem
from rdkit.Chem import AllChem

# Path to the gzipped PDB file
gzipped_pdb_path = "../../data/curated_chembl/alpha_fold_data/HUMAN/AF-P14416-F1-model_v4.pdb.gz"

# Use gzip to open and read the gzipped PDB file
with gzip.open(gzipped_pdb_path, 'rt') as file:
    pdb_data = file.read()

# Convert the PDB data to an RDKit molecule
protein = Chem.MolFromPDBBlock(pdb_data, removeHs=False)

# Do charge calculation (TODO)

# Save the prepared structure to a new PDB file
Chem.MolToPDBFile(protein, "prepared_d2_recept.pdb")

In [38]:
convert_pdb_to_pdbqt("prepared_d2_recept.pdb", "prepared_d2_recept.pdbqt")

File converted: prepared_d2_recept.pdbqt


In [40]:
import os
from vina import Vina

# Filepath for the docking results
docking_results_filepath = "docking_results_generic.pdbqt"

# Delete the existing results file if it exists
if os.path.exists(docking_results_filepath):
    os.remove(docking_results_filepath)

# Initialize Vina
v = Vina()

# Set receptor and ligand
v.set_receptor(rigid_pdbqt_filename="prepared_d2_recept.pdbqt")
v.set_ligand_from_file('generic_ligand.pdbqt')

# Compute Vina maps and perform docking
v.compute_vina_maps(center=[0., 0., 0.], box_size=[30, 30, 30])
print(v.score())
print(v.optimize())

# Docking with exhaustiveness set to 32
v.dock(exhaustiveness=32)

# Write the docking results
v.write_poses(pdbqt_filename=docking_results_filepath)


TypeError: 

PDBQT parsing error: Unknown or inappropriate tag found in flex residue or ligand.
 > ATOM      3  O1  UNL     1      -1.388   0.657   0.371  0.00  0.00    -0.395 OA


In [35]:
import os
from vina import Vina

# Filepath for the docking results
docking_results_filepath = "docking_results_d_block.pdbqt"

# Delete the existing results file if it exists
if os.path.exists(docking_results_filepath):
    os.remove(docking_results_filepath)

# Initialize Vina
v = Vina()

# Set receptor and ligand
v.set_receptor(rigid_pdbqt_filename="prepared_d2_recept.pdbqt")
v.set_ligand_from_file('d_block_ligand.pdbqt')

# Compute Vina maps and perform docking
v.compute_vina_maps(center=[0., 0., 0.], box_size=[30, 30, 30])
print(v.score())
print(v.optimize())

# Docking with exhaustiveness set to 32
v.dock(exhaustiveness=32)

# Write the docking results
v.write_poses(pdbqt_filename=docking_results_filepath)


RuntimeError: Error: file prepared_protein.pdbqt does not exist.